In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para gráficos
plt.style.use('default')
sns.set_palette("husl")


In [32]:
# 1. Descargar datos
print("Descargando datos de DOGE-USD...")
df = yf.download("DOGE-USD", start="2020-01-01", end="2024-12-31")

# Verificar que tenemos datos
if df.empty:
    print("No se pudieron descargar datos. Verifica el símbolo y las fechas.")
else:
    print(f"Datos descargados: {len(df)} registros")
    print(f"Última fecha disponible: {df.index[-1].strftime('%Y-%m-%d')}")


[*********************100%***********************]  1 of 1 completed

Descargando datos de DOGE-USD...
Datos descargados: 1826 registros
Última fecha disponible: 2024-12-30


In [33]:
# 2. Preparar datos
df = df[['Close']].copy()
df.columns = ['Precio_Cierre']

# Crear variables predictoras (features)
df['Dia'] = range(len(df))
df['Media_Movil_7'] = df['Precio_Cierre'].rolling(window=7).mean()
df['Media_Movil_30'] = df['Precio_Cierre'].rolling(window=30).mean()
df['Retorno_1d'] = df['Precio_Cierre'].pct_change()
df['Volatilidad_7d'] = df['Retorno_1d'].rolling(window=7).std()

# Variable objetivo: precio en 6 días
df['Precio_6d_Futuro'] = df['Precio_Cierre'].shift(-6)

# Eliminar filas con NaN
df_clean = df.dropna().copy()

print(f"Datos limpios: {len(df_clean)} registros")


Datos limpios: 1791 registros


In [34]:
# 3. Seleccionar los últimos 30 días para entrenamiento
ultima_fecha = df_clean.index[-1]
fecha_inicio_30d = ultima_fecha - pd.Timedelta(days=30)
df_train = df_clean[df_clean.index >= fecha_inicio_30d].copy()

print(f"Datos de entrenamiento (últimos 30 días): {len(df_train)} registros")


Datos de entrenamiento (últimos 30 días): 31 registros


In [35]:
# 4. Preparar variables para el modelo
features = ['Dia', 'Media_Movil_7', 'Media_Movil_30', 'Retorno_1d', 'Volatilidad_7d']
X = df_train[features]
y = df_train['Precio_6d_Futuro']

# Escalar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [36]:
# 5. Entrenar modelo GLM usando statsmodels.api correctamente
print("\nEntrenando modelo GLM...")
# Añadir constante para el intercepto
X_sm = sm.add_constant(X_scaled)
glm_model = sm.GLM(y, X_sm, family=sm.families.Gaussian())
glm_results = glm_model.fit()

print(glm_results.summary())


Entrenando modelo GLM...
                 Generalized Linear Model Regression Results                  
Dep. Variable:       Precio_6d_Futuro   No. Observations:                   31
Model:                            GLM   Df Residuals:                       25
Model Family:                Gaussian   Df Model:                            5
Link Function:               Identity   Scale:                      0.00032739
Method:                          IRLS   Log-Likelihood:                 83.724
Date:                Thu, 28 Aug 2025   Deviance:                    0.0081849
Time:                        21:38:41   Pearson chi2:                  0.00818
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9992
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3790     

In [37]:
# 6. Hacer predicciones para los próximos 6 días
ultimo_dia = df_train['Dia'].iloc[-1]
ultimos_datos = df_train.iloc[-1:][features].values

# Preparar datos para predicción de 6 días
future_days = []
for i in range(1, 7):
    future_day = ultimos_datos.copy()
    future_day[0, 0] = ultimo_dia + i
    future_days.append(future_day[0])

future_days = np.array(future_days)
future_days_scaled = scaler.transform(future_days)
future_days_sm = sm.add_constant(future_days_scaled)  # Añadir constante

# Hacer predicciones
predicciones_glm = glm_results.predict(future_days_sm)


ValueError: shapes (6,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)